---
title: "Understanding Markov Chains"
author: "Kirtan Gangani"
date: "July 10, 2025" 
categories: [Probability, Statistics]
format:
  html:
    toc: true
    code-fold: false
    code-copy: true
jupyter: python3
---

# Introduction: What is a Markov Chain?

Markov chains, named after Andrey Markov is a powerful mathematical model used to show a sequence of events where the probability of each event depends *only* on the state achieved in the previous event. It doesn't care about the entire history that led to the current state, just the current state itself. This unique "memoryless" property makes them surprisingly versatile and useful for modeling a wide range of real-world phenomena.

# The Core Concepts: States, Transitions, and The Markov Property

To understand Markov Chains, we need to understand some concepts:

## States

**States** are all the possible "situations" or "conditions" our system can be in. For example:   

* **Weather:** Sunny, Cloudy, Rainy  
* **A Stock Price:** Up, Down, Stagnant  
* **Your Mood:** Happy, Neutral, Sad  

## Transitions and Probabilities

A **transition** is simply the movement from one state to another. For example, the weather changing from "Sunny" to "Cloudy."

Each transition comes with a **transition probability**, which is the likelihood of moving from a current state to a next state. These probabilities are always between 0 and 1.

**Note**: For any given state in a system, the sum of the probabilities of all possible transitions originating from that state (including the probability of transitioning back to itself) must always equal 1. This holds true regardless of the total number of states in the system.

## The Markov Property

The **Markov Property** means that the probability of moving to a future state depends only on the **current state**, and not on any of the states that came before it.

Imagine you're playing Monopoly and you land on a specific square, it doesn't matter if you got there by rolling 6 and 2, or by rolling 5 and 3. All that matters is that you are currently on that square. This "forgetfulness" of past events is what makes Markov Chains unique and, surprisingly, easier to model.

# Initial State Probability / Prior Probability

**Initial state probability distribution** gives information about where our system begins, sometimes also referred to as the **prior probability** or **starting probability vector**.

This vector represents the likelihood of the system being in each of its possible states at the very beginning (at time $t=0$).

Let's denote the initial state probability vector as $\pi_0$. If we have $N$ states, then $\pi_0$ would be a row vector:

$$\pi_0 = \begin{pmatrix} P(\text{State}_1 \text{ at } t=0) & P(\text{State}_2 \text{ at } t=0) & \dots & P(\text{State}_N \text{ at } t=0) \end{pmatrix}$$

Where $P(\text{State}_i \text{ at } t=0)$ is the probability of being in State $i$ at the initial time.

**Key characteristics of the initial state probability vector:**

* **Non-negative:** Each probability must be greater than or equal to 0.
* **Sums to 1:** The sum of all probabilities in the vector must equal 1, as the system *must* be in one of the states at the start.

**Example using our weather states:**

Suppose we want to start our weather prediction. We might have prior knowledge about today's weather:

* There's a 70% chance it's Sunny.
* There's a 20% chance it's Cloudy.
* There's a 10% chance it's Rainy.

In this case, our initial state probability vector $\pi_0$ would be:

$$\pi_0 = \begin{pmatrix} 0.7 & 0.2 & 0.1 \end{pmatrix}$$

If we are absolutely certain about the starting state (e.g., we know for a fact it's Sunny today), then the initial state vector would have a 1 in the position corresponding to that state and 0s elsewhere:

$$\text{If starting state is definitively Sunny:} \quad \pi_0 = \begin{pmatrix} 1 & 0 & 0 \end{pmatrix}$$

This initial distribution is the first piece of information, along with the transition matrix, that allows us to forecast the probabilities of being in various states at future time steps.

# The Transition Matrix: A Numerical Map

While state diagrams are great for visualization, mathematically, we represent all these transition probabilities in a **transition matrix** (often denoted as $P$).

This matrix is essentially a table where:  

* Each **row** represents the **current state** (where you're coming *from*).  
* Each **column** represents the **next state** (where you're going *to*).  
* The value at the intersection of a row and a column is the probability of transitioning from the row's state to the column's state.

Using our weather example, the transition matrix would look like this:

$$
P = \begin{pmatrix}
0.6 & 0.3 & 0.1 \\
0.2 & 0.3 & 0.5 \\
0.4 & 0.1 & 0.5
\end{pmatrix}
$$

Let's represent this in Python using NumPy and Pandas for clear display:

In [22]:
import numpy as np
import pandas as pd

states = ["Sunny", "Cloudy", "Rainy"]

P = np.array([
  # Sunny Cloudy   Rainy
    [0.6,   0.3,   0.1], # Sunny 
    [0.2,   0.3,   0.5], # Cloudy 
    [0.4,   0.1,   0.5]  # Rainy
])

transition_df = pd.DataFrame(P, index=[f"From {s}" for s in states], columns=[f"To {s}" for s in states])
print("Our Weather Transition Matrix (P):")
print(transition_df)

print("\nRow sums should be equal to 1.0.")
print(transition_df.sum(axis=1))

Our Weather Transition Matrix (P):
             To Sunny  To Cloudy  To Rainy
From Sunny        0.6        0.3       0.1
From Cloudy       0.2        0.3       0.5
From Rainy        0.4        0.1       0.5

Row sums should be equal to 1.0.
From Sunny     1.0
From Cloudy    1.0
From Rainy     1.0
dtype: float64


# Visualizing The State Diagram

A great way to understand the flow within a Markov Chain is through a **state diagram**. This visual representation uses circles for states and arrows for transitions, with the transition probabilities labeled on the arrows.

Let's use a simple weather example:

* If it's Sunny today, there's a 60% chance it stays Sunny tomorrow, 30% chance it becomes Cloudy, and 10% chance it becomes Rainy.
* If it's Cloudy today, there's a 20% chance it becomes Sunny, 30% chance it stays Cloudy, and 50% chance it becomes Rainy.
* If it's Rainy today, there's a 40% chance it becomes Sunny, 10% chance it becomes Cloudy, and 50% chance it stays Rainy.

![](state-diagram.jpg){fig-align="center"}

State diagram. Source: [https://en.proft.me/2014/04/14/how-simulate-markov-chain-r/](https://en.proft.me/2014/04/14/how-simulate-markov-chain-r/)

# Types of Markov Chains

While the core concepts remain the same, Markov Chains can be categorized in a couple of important ways, primarily based on how time and states are handled.

## Discrete-Time vs. Continuous-Time

* **Discrete-Time Markov Chains (DTMC):** In a DTMC, transitions between states occur at fixed, regular intervals or "steps." Think of it like taking a snapshot of the system every hour, day, or year. Our weather example is a perfect DTMC because we're looking at the weather "tomorrow" based on "today." The transition matrix you've seen applies directly to DTMCs.

* **Continuous-Time Markov Chains (CTMC):** In contrast, CTMCs allow transitions to occur at *any* point in time, not just at predefined intervals. The amount of time spent in a particular state before transitioning is a random variable, often modeled by an exponential distribution. While more complex mathematically, CTMCs are used to model systems where events happen asynchronously, like customer arrivals in a queue or radioactive decay.

## Discrete-State vs. Continuous-State

This distinction refers to the nature of the states themselves.

* **Discrete-State Markov Chains:** This is what we've been discussing. The system can only be in a finite (or countably infinite) number of distinct, separate states. Examples include: "Sunny," "Cloudy," "Rainy"; "Up," "Down," "Stagnant" for stock prices; or even the squares on a Monopoly board. All the examples and the transition matrix we've used so far fall into this category.

* **Continuous-State Markov Chains:** In these chains, the state space is continuous, meaning the system can take on any value within a range. For instance, modeling the exact temperature (e.g., 25.3 degrees, 25.31 degrees, etc.) or the precise stock price value over time. These are often more complex and are typically modeled using stochastic differential equations, moving beyond simple transition matrices. For an introductory understanding, focusing on discrete-state chains is appropriate.

# State Sampling / Markov Chain Simulation

While the transition matrix helps us understand the probabilities of moving between states, what if we want to see an actual *sequence* of events unfold according to these probabilities? This is where **state sampling**, also known as **Markov Chain simulation**, comes in.

Imagine we want to predict the weather for the next 10 days based on our weather Markov Chain. We don't want to just know the *probabilities* of it being sunny on day 10; we want to simulate a *possible sequence* of weather conditions.

### Why is State Sampling Important?

* **Understanding Long-Term Behavior:** By simulating the chain for many steps, we can get a sense of its long-term behavior, such as how often it visits certain states, or what the probability of being in a particular state is after a very long time (this is related to the stationary distribution).
* **Approximating Complex Probabilities:** For very complex systems or chains with many states, analytically calculating probabilities for many steps into the future can be computationally intractable. Simulation provides a practical way to approximate these probabilities.
* **Foundation for Markov Chain Monte Carlo (MCMC):** State sampling is the core idea behind **(MCMC)** methods. MCMC is a powerful class of algorithms used extensively in statistics, machine learning, and computational physics to sample from complex probability distributions, especially in Bayesian inference. Instead of directly calculating an intractable distribution, MCMC constructs a Markov chain whose stationary distribution is the target distribution, and then samples from that chain.

### How it Works (Conceptually)

Let's use our weather example to illustrate the process:

1.  **Start in an Initial State:** We pick a starting state. Let's say it's "Sunny" today.
2.  **Consult the Transition Probabilities:** Look at the row in the transition matrix corresponding to the current state ("Sunny"). We see:
    * Sunny to Sunny: 60%
    * Sunny to Cloudy: 30%
    * Sunny to Rainy: 10%
3.  **Randomly Select the Next State:** Based on these probabilities, we "roll the dice" (or use a random number generator) to decide the next state.
    * If our random number (between 0 and 1) is `0.0` to `0.6`, the next state is "Sunny."
    * If it's `0.6` to `0.9`, the next state is "Cloudy."
    * If it's `0.9` to `1.0`, the next state is "Rainy."
4.  **Repeat:** The newly chosen state becomes the "current state," and we repeat steps 2 and 3 for as many steps as we desire.

By repeating this process many times, we generate a **sequence of states** that represents a possible path the system could take. If we run many such simulations, we can estimate the likelihood of various outcomes or the average behavior of the system over time.

Let's simulate a short sequence of weather for 5 days, starting with Sunny:

In [23]:
def simulate_markov_chain(initial_state_idx, P, num_steps, states_names):
    current_state_idx = initial_state_idx
    path = [states_names[current_state_idx]]

    for _ in range(num_steps):
        probabilities = P[current_state_idx, :]

        next_state_idx = np.random.choice(len(states_names), p=probabilities)

        current_state_idx = next_state_idx
        path.append(states_names[current_state_idx])
    return path

num_simulation_days = 5
initial_weather_state = 0
simulated_path = simulate_markov_chain(initial_weather_state, P, num_simulation_days, states)

print(f"Simulated weather path for {num_simulation_days} days, starting Sunny:")
print(simulated_path)

# You can run this cell multiple times to see different simulated paths

Simulated weather path for 5 days, starting Sunny:
['Sunny', 'Rainy', 'Sunny', 'Sunny', 'Cloudy', 'Rainy']


# Real-World Applications

Markov Chains are applied in countless real-world scenarios:

* **Weather Forecasting:** As seen in our example, predicting future weather patterns.
* **Google PageRank:** One of the original algorithms that powered Google's search engine. Web pages were considered states, and links between them were transitions. Pages with a higher "probability" of being navigated to (in the long run) by a random surfer were deemed more important and ranked higher.
* **Natural Language Processing (NLP):** Used in tasks like predicting the next word in a sentence (think predictive text on your phone), speech recognition, and spam filtering.
* **Finance:** Modeling stock price movements, option pricing, and credit risk.
* **Biology:** Analyzing DNA sequences, modeling population dynamics, and disease spread.
* **Games:** Analyzing game theory strategies, or even simple games like "Snakes and Ladders" can be modeled as Markov Chains.
* **Customer Behavior:** Modeling customer journeys through a website or predicting subscription churn.